In [ ]:
#question2
import numpy as np

def calculate_joint_velocities(Jacobian, end_effector_velocities):
    """
    Calculate joint velocities from the Jacobian matrix and end-effector velocities.

    Parameters:
    Jacobian (numpy.ndarray): The Jacobian matrix relating joint velocities to end-effector velocities.
    end_effector_velocities (numpy.ndarray): The desired end-effector Cartesian velocities.

    Returns:
    joint_velocities (numpy.ndarray): The calculated joint velocities.
    """
    # Check if the Jacobian and end-effector velocities have compatible shapes.
    if Jacobian.shape[0] != Jacobian.shape[1] or Jacobian.shape[0] != end_effector_velocities.shape[0]:
        raise ValueError("Invalid dimensions for Jacobian or end-effector velocities.")

    # Calculate joint velocities using the Jacobian.
    joint_velocities = np.linalg.pinv(Jacobian).dot(end_effector_velocities)

    return joint_velocities

# Example usage:
# Define the Jacobian matrix and end-effector velocities.
Jacobian = np.array([
    [0.1, 0.2, 0.3],
    [0.4, 0.5, 0.6],
    [0.7, 0.8, 0.9]
])

end_effector_velocities = np.array([0.1, 0.2, 0.3])

# Calculate joint velocities.
joint_velocities = calculate_joint_velocities(Jacobian, end_effector_velocities)
print("Joint Velocities:", joint_velocities)


In [ ]:
#question5
import math
import numpy as np

def spherical_wrist_inverse_kinematics(R):
    # Extract the rotation matrix R, representing the desired orientation.
    # R should be a 3x3 rotation matrix.

    # Extract the elements of the rotation matrix for easier computation.
    r11, r12, r13 = R[0, 0], R[0, 1], R[0, 2]
    r21, r22, r23 = R[1, 0], R[1, 1], R[1, 2]
    r31, r32, r33 = R[2, 0], R[2, 1], R[2, 2]

    # Calculate the yaw (rotation about the Z-axis) using arctan2.
    yaw = math.atan2(r21, r11)

    # Calculate the pitch (rotation about the Y-axis).
    pitch = math.atan2(-r31, math.sqrt(r32**2 + r33**2))

    # Calculate the roll (rotation about the X-axis).
    roll = math.atan2(r32, r33)

    return yaw, pitch, roll

# Example usage with a rotation matrix representing the desired orientation.
# Replace the values in the rotation matrix accordingly.
R_desired = np.array([[0.866, -0.5, 0],
                     [0.5, 0.866, 0],
                     [0, 0, 1]])

yaw, pitch, roll = spherical_wrist_inverse_kinematics(R_desired)
print("Yaw (Z-axis rotation):", math.degrees(yaw), "degrees")
print("Pitch (Y-axis rotation):", math.degrees(pitch), "degrees")
print("Roll (X-axis rotation):", math.degrees(roll), "degrees")


In [ ]:
#question6-a
import numpy as np
import sympy as sp

# Define symbolic variables
q1, q2, q3, q1_dot, q2_dot, q3_dot, q1_ddot, q2_ddot, q3_ddot = sp.symbols('q1 q2 q3 q1_dot q2_dot q3_dot q1_ddot q2_ddot q3_ddot')
t1, t2, t3 = sp.symbols('t1 t2 t3')

# Robot parameters (inertias, lengths, etc.)
# Replace with appropriate values for your robot.
m1, m2, m3 = 1.0, 1.0, 1.0
l1, l2, l3 = 1.0, 1.0, 1.0
I1, I2, I3 = 0.1, 0.1, 0.1
g = 9.81

# Forward kinematics
x = l1 * sp.cos(q1) + l2 * sp.cos(q1 + q2) + l3 * sp.cos(q1 + q2 + q3)
y = l1 * sp.sin(q1) + l2 * sp.sin(q1 + q2) + l3 * sp.sin(q1 + q2 + q3)

# Define Lagrangian for the robot
T = 0.5 * (m1 * (l1 * q1_dot)**2 + I1 * q1_dot**2 +
            m2 * ((l1 * q1_dot + l2 * q2_dot * sp.cos(q2))**2) + I2 * (q1_dot + q2_dot)**2 +
            m3 * ((l1 * q1_dot + l2 * q2_dot * sp.cos(q2) + l3 * q3_dot * sp.cos(q2 + q3))**2) + I3 * (q1_dot + q2_dot + q3_dot)**2)
U = m1 * g * l1 * sp.sin(q1) + m2 * g * (l1 * sp.sin(q1) + l2 * sp.sin(q1 + q2)) + m3 * g * (l1 * sp.sin(q1) + l2 * sp.sin(q1 + q2) + l3 * sp.sin(q1 + q2 + q3))
L = T - U

# Calculate the equations of motion
eom = [sp.Eq(sp.diff(L, q_i) - sp.diff(sp.diff(L, q_i_dot), q_i_dot) - t_i, 0) for q_i, q_i_dot, t_i in zip([q1, q2, q3], [q1_dot, q2_dot, q3_dot], [t1, t2, t3])]

# Define initial conditions and small torques
initial_conditions = {q1: 0.1, q2: 0.1, q3: 0.1, q1_dot: 0.01, q2_dot: 0.01, q3_dot: 0.01}
torques = {t1: 0.01, t2: 0.01, t3: 0.01}

# Solve for joint accelerations (q1_ddot, q2_ddot, q3_ddot)
joint_accelerations = sp.solve(eom, [q1_ddot, q2_ddot, q3_ddot])

# Simulate the robot's dynamic behavior
time_step = 0.01  # Time step for simulation
num_steps = 100    # Number of time steps

q1_vals = [initial_conditions[q1]]
q2_vals = [initial_conditions[q2]]
q3_vals = [initial_conditions[q3]]

for _ in range(num_steps):
    # Update joint velocities using small torques
    q1_dot_new = initial_conditions[q1_dot] + joint_accelerations[q1_ddot] * time_step + torques[t1] * time_step
    q2_dot_new = initial_conditions[q2_dot] + joint_accelerations[q2_ddot] * time_step + torques[t2] * time_step
    q3_dot_new = initial_conditions[q3_dot] + joint_accelerations[q3_ddot] * time_step + torques[t3] * time_step

    # Update joint positions
    q1_new = initial_conditions[q1] + q1_dot_new * time_step
    q2_new = initial_conditions[q2] + q2_dot_new * time_step
    q3_new = initial_conditions[q3] + q3_dot_new * time_step

    # Append values for plotting
    q1_vals.append(q1_new)
    q2_vals.append(q2_new)
    q3_vals.append(q3_new)

    # Update initial conditions for the next time step
    initial_conditions[q1] = q1_new
    initial_conditions[q2] = q2_new
    initial_conditions[q3] = q3_new
    initial_conditions[q1_dot] = q1_dot_new
    initial_conditions[q2_dot] = q2_dot_new
    initial_conditions[q3_dot] = q3_dot_new

# Print joint positions
print("Joint Positions:")
print("q1:", q1_vals)
print("q2:", q2_vals)
print("q3:", q3_vals)


In [ ]:
#question6-b
import numpy as np
import matplotlib.pyplot as plt

# Define robot parameters, dynamics, and initial conditions as in the previous code.

# Desired trajectory for joint positions (example)
def desired_trajectory(time):
    q1_desired = np.sin(time)
    q2_desired = 0.5 * np.cos(2 * time)
    q3_desired = -0.2 * np.sin(3 * time)
    return q1_desired, q2_desired, q3_desired

# Small stochastic disturbances (example)
def generate_disturbance():
    return np.random.normal(0, 0.01, 3)

# Control gains (example)
kp = 10.0
kd = 1.0

# Lists to store data for plotting
time_steps = []
q1_vals = []
q2_vals = []
q3_vals = []

# Simulation parameters
time_step = 0.01
num_steps = 100

# Initial conditions
q1 = 0.1
q2 = 0.1
q3 = 0.1
q1_dot = 0.01
q2_dot = 0.01
q3_dot = 0.01

for step in range(num_steps):
    time = step * time_step
    q1_desired, q2_desired, q3_desired = desired_trajectory(time)
    disturbance = generate_disturbance()

    # Calculate errors and control torques
    error_q1 = q1_desired - q1
    error_q2 = q2_desired - q2
    error_q3 = q3_desired - q3

    control_torque = kp * np.array([error_q1, error_q2, error_q3]) - kd * np.array([q1_dot, q2_dot, q3_dot]) + disturbance

    # Update joint velocities
    q1_dot += (control_torque[0] / I1) * time_step
    q2_dot += (control_torque[1] / I2) * time_step
    q3_dot += (control_torque[2] / I3) * time_step

    # Update joint positions
    q1 += q1_dot * time_step
    q2 += q2_dot * time_step
    q3 += q3_dot * time_step

    # Append data for plotting
    time_steps.append(time)
    q1_vals.append(q1)
    q2_vals.append(q2)
    q3_vals.append(q3)

# Plot the joint positions
plt.figure(figsize=(10, 6))
plt.plot(time_steps, q1_vals, label='Joint 1')
plt.plot(time_steps, q2_vals, label='Joint 2')
plt.plot(time_steps, q3_vals, label='Joint 3')
plt.xlabel('Time (seconds)')
plt.ylabel('Joint Positions')
plt.legend()
plt.title('Joint Positions with Simple Independent Joint Control')
plt.grid(True)
plt.show()


In [ ]:
#question1
import math

def inverse_kinematics(x, y, link1_length, link2_length):
    # Calculate the distance from the origin to the end-effector position.
    distance = math.sqrt(x**2 + y**2)

    # Calculate the angle theta2 using the law of cosines.
    cos_theta2 = (link1_length**2 + link2_length**2 - distance**2) / (2 * link1_length * link2_length)
    theta2 = math.acos(cos_theta2)

    # Calculate the angle theta1 using trigonometric functions.
    alpha = math.atan2(y, x)
    beta = math.acos((link1_length**2 + distance**2 - link2_length**2) / (2 * link1_length * distance))
    theta1 = alpha - beta

    return theta1, theta2

# Example usage with representative numerical values:
target_x = 3.0  # Desired x-coordinate of the end-effector.
target_y = 4.0  # Desired y-coordinate of the end-effector.
link1_length = 2.0  # Length of the first link.
link2_length = 3.0  # Length of the second link.

joint_angles = inverse_kinematics(target_x, target_y, link1_length, link2_length)
print("Joint Angles (theta1, theta2):", joint_angles)
